v0.2, output txt. To copy the txt to data.sql at java backend. 

The website we are scraping: https://open.bccampus.ca/browse-our-collection/find-open-textbooks/
Total number of books: 382


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import openpyxl

In [21]:
chrome_options = Options()
#stops the browser from popping up when this program runs 
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path="chromedriver", chrome_options=chrome_options)

c:\Users\artac\anaconda3\envs\condaEx\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [22]:
CATALOGUE_LINK_1ST_HALF = 'https://open.bccampus.ca/browse-our-collection/find-open-textbooks/?start=' 
list_number = 0 
CATALOGUE_LINK_2ND_HALF = '&subject=&contributor=&searchTerm=&keyword=' 

catalogue_all_links = [] 
#prepare links of catalogue to iterate through
for i in range(0, 390, 10): 
    catalogue_all_links.append(CATALOGUE_LINK_1ST_HALF + str(i) + CATALOGUE_LINK_2ND_HALF)

In [23]:
list_of_links_of_all_books = [] 
BOOK_LINK_1ST_HALF = 'https://open.bccampus.ca/browse-our-collection/find-open-textbooks/'
#iterate through all 38 pages of catalogues
for link in catalogue_all_links:
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    #get links of each books in the page
    for link in soup.findAll('a', href=True):
        if ('uuid' in link.attrs.get('href')): 
            list_of_links_of_all_books.append(BOOK_LINK_1ST_HALF + link.attrs.get('href'))

#remove duplicates 
list_of_links_of_all_books = list(dict.fromkeys(list_of_links_of_all_books))

In [24]:
def formaturl(url):
    if not re.match('(?:http|ftp|https):', url):
        return 'http:{}'.format(url)
    return url

def setGenre(link): 
    if ('Biological' or 'Astronomy' or 'Biology' or 'Chemistry' or 'Physics' or 'Science') in link: 
        return 'Biological/Physical Sciences'
    elif ('Business' or 'Career' or 'College' or 'Accounting' or 'Business' or 'Commerce' or 'Entrepreneurship' or 'Finance' or 'Human' or 'Management' or 'Non-Profits') in link: 
        return 'Business'
    elif ('Computer' or 'Algorithms' or 'Computer' or 'Databases' or 'Systems' or 'Web') in link: 
        return 'Computer Sciences' 
    elif ('Humanities' or 'English' or 'Gender' or 'History' or 'Lingustics' or 'Philosophy' or 'Spanish') in link:
        return 'Humanities'
    elif ('Math' or 'Algebra' or 'Calculus' or 'Statistics') in link: 
        return 'Math/Stats'

    
    else: 
        return 'Others'


def escapeChars(string):
    return string.translate(str.maketrans({"'": r"\'",
                                            "\r": r"",
                                            "\n": r""}))
#                                       "-":  r"\-",
                                        #   "]":  r"\]",
                                        #   "\\": r"\\",
                                        #   "^":  r"\^",
                                        #   "$":  r"\$",
                                        #   "*":  r"\*",
                                        #   ".":  r"\."

#build sql statement for data.sql 
sql = 'INSERT INTO `book` (`book_id`, `author`, `description`, `genre`, `link`, `photo`, `publish_date`, `title`) VALUES\n'

df = pd.DataFrame(columns =['book-id', 'title', 'genre', 'description'])
#iterate through links to each book to get information of books and store them in the sql statement
#sql statement requires manual cleaning at the end ',' 
number = 1 

for link in list_of_links_of_all_books: 
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    #parse a webpage for book details 
    bookDetailsHTML = soup.find(attrs={'itemtype':'https://schema.org/Book'})

    author = escapeChars(bookDetailsHTML.find(attrs={'itemprop':'author copyrightHolder'}).text)

    description = escapeChars(bookDetailsHTML.find(attrs={'itemprop':'description'}).text)
    description_split = re.findall(r'\w+', description)[:15]
    description = " ".join(description_split)
    description = description + " ..."

    genre = setGenre(soup.find(attrs={'itemprop':"about"}).text)

    photo_filename = 'book_' + str(number) + '.jpg'

    #substring to keep date only
    publish_date = bookDetailsHTML.find(attrs={'itemprop':'datePublished'}).attrs.get('content')[:10]

    name = escapeChars(bookDetailsHTML.find(attrs={'itemprop':'name'}).text )


    # #download image file. photo string is image filename. 
    # image_url = soup.find(attrs={'itemprop':'image'}).attrs.get('src')
    # image_url = formaturl(image_url)
    # img_data = requests.get(image_url).content
    # with open('book_' + str(number) + '.jpg', 'wb') as handler:
    #     handler.write(img_data)

    #check genre from subject areas 
    # print('subject area text:' + soup.find(attrs={'itemprop':"about"}).text)
    # print('genre appended:' + setGenre(soup.find(attrs={'itemprop':"about"}).text))

    #(`book_id`, `author`, `description`, `genre`, `link`, `photo`, `publish_date`, `title`),\n'
    sql += '(' + str(number) + ','
    sql += ' \'' + author + '\','
    sql += ' \'' + description + '\','
    sql += ' \'' + genre + '\','
    sql += ' \'' + link + '\','
    sql += ' \'' + photo_filename + '\','
    sql += ' \'' + publish_date + '\','
    sql += ' \'' + name + '\''
    sql += '),\n'

    #save required data to excel for textbook recommendation
    df = df.append({'book-id' : number, 'title' : name, 'genre' : genre, 'description' : description}, ignore_index=True)

    print('link ' + str(number) + ' done') 
    number = number + 1 

link 1 done
link 2 done
link 3 done
link 4 done
link 5 done
link 6 done
link 7 done
link 8 done
link 9 done
link 10 done
link 11 done
link 12 done
link 13 done
link 14 done
link 15 done
link 16 done
link 17 done
link 18 done
link 19 done
link 20 done
link 21 done
link 22 done
link 23 done
link 24 done
link 25 done
link 26 done
link 27 done
link 28 done
link 29 done
link 30 done
link 31 done
link 32 done
link 33 done
link 34 done
link 35 done
link 36 done
link 37 done
link 38 done
link 39 done
link 40 done
link 41 done
link 42 done
link 43 done
link 44 done
link 45 done
link 46 done
link 47 done
link 48 done
link 49 done
link 50 done
link 51 done
link 52 done
link 53 done
link 54 done
link 55 done
link 56 done
link 57 done
link 58 done
link 59 done
link 60 done
link 61 done
link 62 done
link 63 done
link 64 done
link 65 done
link 66 done
link 67 done
link 68 done
link 69 done
link 70 done
link 71 done
link 72 done
link 73 done
link 74 done
link 75 done
link 76 done
link 77 done
link 78 

In [25]:
print(len(list_of_links_of_all_books))

382


In [26]:
with open('sql.txt' , 'w', encoding="utf-8") as f: 
    f.write(sql)

In [27]:
df.head()

,book-id,title,genre,description
0,1,University Physics - Volume 1 (OpenStax),Biological/Physical Sciences,University Physics is a three volume collectio...
1,2,Conservation Biology in Sub-Saharan Africa,Biological/Physical Sciences,Easy to read this lucid and accessible textboo...
2,3,Psychology of Language,Others,In this textbook students are introduced to th...
3,4,Adult Literacy Fundamental Mathematics: Book 1...,Others,This is book one in a six book series on funda...
4,5,Introduction to Drafting and AutoCAD 2D,Others,Introduction to Drafting and AutoCAD 2D was wr...


In [28]:
df.to_excel('textBooksData.xlsx', index=False, header=True)